In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tkinter import *
from PIL import Image, ImageTk
# import time
# import argparse
import os
import random

In [2]:
def sorted_image_path(file_path):
    file_type = ".jpg"
    image_paths = []

    for subs, dirs, files in os.walk(file_path):
        for file in files:
            path = subs + file
            image_paths.append(path)
    image_paths.sort()
    return image_paths

In [3]:
def load_image():
    global sorted_images, img_cur_idx
    img = cv2.imread(sorted_images[img_cur_idx])
    img_cur_idx += 1
    return img

In [4]:
def is_inside(o,i):
    ox,oy,ow,oh = o
    ix,iy,iw,ih = i
    return ox>ix and oy>iy and ox+ow<ix+iw and oy+oh<iy+ih

In [5]:
def draw_person(image,person, color):
    x,y,w,h = person
    cv2.rectangle(image, (x,y), (x+w, y+h),color,3)

In [6]:
# Using Hog to find the pedestrians list & return the list
def findlist(img):
    global hog
    img_gau = cv2.GaussianBlur(img, (9, 9), 0)
    grey = cv2.cvtColor(img_gau, cv2.COLOR_RGB2GRAY)
    found, w = hog.detectMultiScale(grey, winStride=(10,10))
    found_list = []
    for ri, r in enumerate(found):
        flag = 0
        for qi, q in enumerate(found):
            if ri != qi and is_inside(r,q):
                flag = 1
        if flag == 0:
            found_list.append(r)
    return found_list

In [7]:
# Get the starter mouse touch point
def get_mouse_posn(event):
    global topy, topx
    
    topx, topy = event.x, event.y
    
# Get the final mouse touch point & update select status
def update_sel_rect(event):
    global rect_id
    global topy, topx, botx, boty
    global select

    botx, boty = event.x, event.y
    select = True

In [8]:
# Compute the iou score and return it
def cal_bounding_boxes_iou(b1, b2):
    x1,y1,w1,h1 = b1
    x2,y2,w2,h2 = b2

    # determine the coordinates of the intersection rectangle
    x_left = max(x1, x2)
    y_top = max(y1, y2)
    x_right = min(x1+w1, x2+w2)
    y_bottom = min(y1+h1, y2+h2)

    if x_right < x_left or y_bottom < y_top:
        return 0
    
    # compute the area of the bouding boxes when boxes overlap
    b1_area = (w1 * h1)
    b2_area = (w2 * h2)
    
    # compute the area of the intersection between bounding boxes
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the intersection over union between bounding boxes
    iou = intersection_area / float(b1_area + b2_area - intersection_area)
    return iou

In [9]:
# Conver list to string
def list_to_str(list):
    return ','.join(str(i) for i in list)

In [10]:
# Find nearest pedestrian and check whether they unique or not
def find_nearest(center, pre_center_list):
    min_distance = 50
    nearest_c = 0

    for cc in pre_center_list:
        temp_distance = ((center['center'][0] - cc['center'][0]) ** 2 + (center['center'][1] - cc['center'][1]) ** 2) ** 0.5
        if temp_distance < min_distance:
            min_distance = ((center['center'][0] - cc['center'][0]) ** 2 + (center['center'][1] - cc['center'][1]) ** 2) ** 0.5
            nearest_c = cc
    return nearest_c

In [11]:
def bounding_box(img, found_list):
    global pre_center_list, line_list,total_num_ped,total_ped_list,total_ped_dict
    curr_center_list = []
    pedestrian_number = 0
    ped_in_group_count = 0
    group_formation_count = 0
    group_destruction_count = 0
    new_ped_count = 0
    for index, person in enumerate(found_list):
        x, y, w, h = person
        pedestrian_number += 1
        center = (int(x), int(y+h))
        center_dict = {'center':center}
        nn = find_nearest(center_dict, pre_center_list) 
        person_color = []
        if nn == 0:
            new_ped_count += 1
            rand_color = random.choices(range(256), k=3)
            person_color = rand_color
            person_dic = {'id': index, 'center': center, 'area': w*h,'color':rand_color,'inGroup': 0, 'inFrame': 1}
            curr_center_list.append({'center':center,'color':rand_color})
            total_ped_dict[list_to_str(rand_color)] = person_dic
            total_num_ped += 1
            total_ped_list.append(person_dic)
        else:
            line_list.append({'color':nn['color'],'line':[nn['center'], center]})
            curr_center_list.append({'center':center,'color':nn['color']})
            person_color = nn['color']
        #### 3.1 & 3.2
        dic_key = list_to_str(person_color)
        person_dic = total_ped_dict.get(dic_key)
        group_flag = 0
        for p in found_list:
            if(np.array_equal(p,person)):
                continue
            else:
                iou = cal_bounding_boxes_iou(person, p)
                if(iou > 0.13):
                    if (group_flag == 0):
                        if (person_dic['inGroup'] == 0):
                            group_formation_count +=1
                        person_dic.update({'inGroup': 1})
                        total_ped_dict[dic_key] = person_dic
                        ped_in_group_count +=1
                        group_flag = 1
        if (group_flag == 0):
            if (person_dic['inGroup'] == 1):
                group_destruction_count +=1
            person_dic.update({'inGroup': 0})
            total_ped_dict[dic_key] = person_dic
        ### 3.1 & 3.2
        draw_person(img, person,person_color)
    #if if_next != 0:
    for line in line_list:
        exist = 0
        #if find_nearest(pre, curr_center_list) == 0:
        for curr in curr_center_list:
            if line['color'] == curr['color']:
                exist = 1
        if exist == 1:
            cv2.line(img, line['line'][0], line['line'][1], line['color'],thickness=3)
    leave_count = 0
    for p in pre_center_list:
        exist2 = 0
        for curr in curr_center_list:
            if p['color'] == curr['color']:
                exist2 = 1
        if exist2 == 0:
            leave_count += 1
            
    pre_center_list = [_ for _ in curr_center_list]
    cv2.putText(img, f'The number of pedestrain is {len(curr_center_list)}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    cv2.putText(img, f'Total number of pedestrain is {total_num_ped}', (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    cv2.putText(img, f'New pedestrian entering {new_ped_count}', (10,120), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    cv2.putText(img, f'Num of Pedestrian leaving {leave_count}', (10,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    cv2.putText(img, f'Number of pedstrain walk as group: {ped_in_group_count}; walk as alone: {len(found_list)-ped_in_group_count}', (10, 1020), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    cv2.putText(img, f'Group formation: {group_formation_count} pedestrain(s) join. Group destruction: {group_destruction_count} pedestrain(s) leave.', (10, 1050), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    return img

In [12]:
# Loop all the image from the file path
def play_video():
    global play_output_images, sorted_images, stop
    stop = False
    while img_cur_idx < len(sorted_images) and not stop:
        image = load_image()
        found_list = findlist(image)
        new_image = bounding_box(image, found_list)
        new_image = cv2.resize(new_image,dsize=(int(org_w/scale_size),int(org_h/scale_size)),fx=1,fy=1,interpolation=cv2.INTER_LINEAR)
        play_output_images.append(new_image)
        PIL_image = Image.fromarray(new_image)
        photo = ImageTk.PhotoImage(PIL_image)
        
        image_label.configure(image=photo)
        image_label.image = photo

        my_window.update_idletasks()

In [13]:
# Check the bouding box is it contain in the selection area
def contains_bounding_box(sel_b, b):
    selx, sely, selw, selh = sel_b
    x, y, w, h = b
    if (selx <= x and sely <= y and (selx+selw) >= (x+w) and (sely+selh) >= (y+h)):
        return True
    return False

In [14]:
# Check pedestrian is it in the selection area
def selected_ped_in_the_box(found_list, select_box):
    selected_found_list = []
    for p in found_list:
        if (contains_bounding_box(select_box, p)):
            selected_found_list.append(p)
            continue
        iou = cal_bounding_boxes_iou(select_box, p)
        if(iou > 0.075):
            selected_found_list.append(p)
    return selected_found_list

In [15]:
# Get the next image from the file path
def next_image():
    global topy, topx, botx, boty, next_output_images, select
    image = load_image()
    found_list = findlist(image)
    new_image = bounding_box(image, found_list)
    if select is True:
        img_selected = cv2.rectangle(image, (topx*scale_size,topy*scale_size), (botx*scale_size, boty*scale_size),(255,255,255),2)
        select_box = [topx*scale_size, topy*scale_size, (botx-topx)*scale_size, (boty-topy)*scale_size]
        selected_found_list = selected_ped_in_the_box(found_list, select_box)
        cv2.putText(new_image, f'Number of pedestrain within region is {len(selected_found_list)}', (10, 990), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
    next_output_images.append(new_image)
    new_image = cv2.resize(new_image,dsize=(int(org_w/scale_size),int(org_h/scale_size)),fx=1,fy=1,interpolation=cv2.INTER_LINEAR)
    PIL_image = Image.fromarray(new_image)
    photo = ImageTk.PhotoImage(PIL_image)

    image_label.configure(image=photo)
    image_label.image = photo

    my_window.update_idletasks()

In [16]:
# Initial HOG
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [17]:
# Enter image path here
file_path = "step_images/train/STEP-ICCV21-02/"

In [18]:
# Main - Open the Tkinter GUI
sorted_images = sorted_image_path(file_path)
play_output_images = []
next_output_images = []
img_cur_idx = 0
total_ped_dict = {}
ini_image = load_image()
scale_size = 2
org_h, org_w, l = ini_image.shape
ini_image = cv2.resize(ini_image,dsize=(int(org_w/scale_size),int(org_h/scale_size)),fx=1,fy=1,interpolation=cv2.INTER_LINEAR)
image_shape = ini_image.shape
my_window = Tk()

my_window.title("Track Pedestrians")
my_window.geometry(f'{image_shape[1]+200}x{image_shape[0]}')

ini_PIL_image = Image.fromarray(ini_image)
ini_photo = ImageTk.PhotoImage(image=ini_PIL_image)

stop = False
selected_cell = 0
total_distance = 0
frame = 0
select = False
selected_cell_sequence = []
pre_center_list = []
line_list = []
total_num_ped = 0

total_ped_list = []

topx, topy, botx, boty = 0, 0, 0, 0
rect_id = None

image_label = Label(my_window, image=ini_photo)
image_label.place(x=0, y=0)
image_label.bind('<Button-1>', get_mouse_posn)
image_label.bind('<B1-Motion>', update_sel_rect)
Button(my_window, text='next image', command=next_image, font=("Arial", 20)).place(x=image_shape[1]+30, y=340)
Button(my_window, text='play video', command=play_video, font=("Arial", 20)).place(x=image_shape[1]+30, y=410)
my_window.mainloop()

In [19]:
def construct_video(images, video_name):
    assert images, "the list is empty"
    h, w, l = images[0].shape
    size = (w, h)
    video_output = cv2.VideoWriter(video_name,cv2.VideoWriter_fourcc(*'DIVX'), 4, size)
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        video_output.write(img)
    video_output.release()

In [20]:
# Export play video button results as vide0
# Note: if it didn't run the play video button it will return a AssertionError
construct_video(play_output_images, "Train Set - Track Pedestrians - play video.avi")

AssertionError: the list is empty

In [ ]:
# Export next video button results as vide0
# Note: if it didn't run the next video button it will return a AssertionError
construct_video(next_output_images, "Train Set - Track Pedestrians - next video.avi")